# Running a Simple MapReduce Example with Ray Core

In this example we're discussing how easy it can be to use Ray to implement MapReduce, a significant milestone in distributed computing.
Many popular big data technologies, such as Hadoop, are built upon this programming model, and it is worth discussing in relation to Ray.
To illustrate its use, we will use a simple example of counting word occurrences across multiple documents.
While this may seem like a straightforward task when working with a small number of documents,
it becomes more complex when dealing with a large corpus, requiring the use of multiple compute nodes to process the data.
This is a commonly used example in distributed computing and is worth learning about.
The approach involves three simple steps:

1. Use a set of documents as the input and apply a specified function to transform or "map" each element within them (such as the words contained in the documents).
   This map phase will produce key-value pairs, where the key represents an element in the document and the value is a metric calculated for that element.
   In this particular case, the goal is to count the number of times each word appears in a document,
   so the map function will output the pair `(word, 1)` every time a word is encountered to show that it has been found once.
2. All the outputs from the map phase are collected and organized based on their key.
   This may involve transferring data between different nodes, as the same key could potentially be found on multiple compute nodes.
   This process is commonly referred to as the shuffle phase.
   As an example, if the map phase produces four occurrences of the pair `(word, 1)`, the shuffle phase will ensure that all occurrences of
   the same word are located on the same node.
3. The reduce phase is so named because it aggregates or combines the elements from the shuffle step.
   Using the example provided, the final count of a word's occurrences is obtained by adding up all the occurrences on each node.
   For example, four instances of `(word, 1)` would be combined to result in a final count of `word: 4`.

MapReduce is named after the first and last stages of the process, but the middle stage is just as crucial.
These phases may appear straightforward, but their strength lies in the ability to run them concurrently on multiple machines.
An example of using the three MapReduce phases on a set of documents divided into three parts is shown in this figure:


![Simple Map Reduce](https://raw.githubusercontent.com/maxpumperla/learning_ray/main/notebooks/images/chapter_02/map_reduce.png)

## Loading Data

We will be using Python to implement the MapReduce algorithm for our word-count purpose and utilizing Ray to parallelize the computation.
To better understand what we are working with, we will begin by loading some example data:

In [13]:
import subprocess
zen_of_python = subprocess.check_output(["python", "-c", "import this"])
corpus = zen_of_python.split()

num_partitions = 3
chunk = len(corpus) // num_partitions
partitions = [
    corpus[i * chunk: (i + 1) * chunk] for i in range(num_partitions)
]

We will be using the Zen of Python, a collection of guidelines from the Python community, as our data for this exercise.
The Zen of Python can be accessed by typing "import this" in a Python session and is traditionally hidden as an "Easter egg."
While it is beneficial for Python programmers to read these guidelines, for the purposes of this exercise,
we will only be counting the number of words contained within them.
To do this, we will divide the Zen of Python into three separate "documents" by treating each line as a separate entity
and then splitting it into these partitions.

## Mapping our Data

To determine the map phase, we require a map function that we will utilize on each document.
In this particular scenario, we want to output the pair `(word, 1)` for every word found in a document.
For basic text documents that are loaded as Python strings, this process appears as follows.

In [14]:
def map_function(document):
    for word in document.lower().split():
        yield word, 1

We will use the apply_map function on a large collection of documents by marking it as a task in Ray using the `@ray.remote` decorator.
When we call `apply_map`, it will be applied to three sets of document data (`num_partitions=3`).
The `apply_map` function will return three lists, one for each partition.
We do this so that Ray can rearrange the results of the map phase and distribute them to the appropriate nodes for us.

In [15]:
import ray

@ray.remote
def apply_map(corpus, num_partitions=3):
    map_results = [list() for _ in range(num_partitions)]
    for document in corpus:
        for result in map_function(document):
            first_letter = result[0].decode("utf-8")[0]
            word_index = ord(first_letter) % num_partitions
            map_results[word_index].append(result)
    return map_results

For text corpora that can be stored on a single machine, it is unnecessary to use the map phase.
However, when the data needs to be divided across multiple nodes, the map phase becomes useful.
In order to apply the map phase to our corpus in parallel, we use a remote call on `apply_map`, just like we have done in previous examples.
The main difference now is that we also specify that we want three results returned (one for each partition) using the num_returns argument.

In [16]:
map_results = [
    apply_map.options(num_returns=num_partitions)
    .remote(data, num_partitions)
    for data in partitions
]

for i in range(num_partitions):
    mapper_results = ray.get(map_results[i])
    for j, result in enumerate(mapper_results):
        print(f"Mapper {i}, return value {j}: {result[:2]}")

Mapper 0, return value 0: [(b'of', 1), (b'is', 1)]
Mapper 0, return value 1: [(b'python,', 1), (b'peters', 1)]
Mapper 0, return value 2: [(b'the', 1), (b'zen', 1)]
Mapper 1, return value 0: [(b'unless', 1), (b'in', 1)]
Mapper 1, return value 1: [(b'although', 1), (b'practicality', 1)]
Mapper 1, return value 2: [(b'beats', 1), (b'errors', 1)]
Mapper 2, return value 0: [(b'is', 1), (b'is', 1)]
Mapper 2, return value 1: [(b'although', 1), (b'a', 1)]
Mapper 2, return value 2: [(b'better', 1), (b'than', 1)]


## Shuffling and Reducing our Data

We can make it so that all pairs from the `j`-th return value end up on the same node for the reduce phase.
Let’s discuss this phase next.
In the reduce phase we can create a dictionary that sums up all word occurrences on each partition:

In [17]:
@ray.remote
def apply_reduce(*results):
    reduce_results = dict()
    for res in results:
        for key, value in res:
            if key not in reduce_results:
                reduce_results[key] = 0
            reduce_results[key] += value

    return reduce_results

We can take the j-th return value from each mapper and send it to the j-th reducer using the following method.
It's important to note that this code works for larger datasets that don't fit on one machine because we are passing references
to the data using Ray objects rather than the actual data itself.
Both the map and reduce phases can be run on any Ray cluster and the data shuffling is also handled by Ray.

In [18]:
outputs = []
for i in range(num_partitions):
    outputs.append(
        apply_reduce.remote(*[partition[i] for partition in map_results])
    )

counts = {k: v for output in ray.get(outputs) for k, v in output.items()}

sorted_counts = sorted(counts.items(), key=lambda item: item[1], reverse=True)
for count in sorted_counts:
    print(f"{count[0].decode('utf-8')}: {count[1]}")

is: 10
better: 8
than: 8
the: 6
to: 5
of: 3
although: 3
be: 3
unless: 2
one: 2
if: 2
implementation: 2
idea.: 2
special: 2
should: 2
do: 2
may: 2
a: 2
never: 2
way: 2
explain,: 2
ugly.: 1
implicit.: 1
complex.: 1
complex: 1
complicated.: 1
flat: 1
readability: 1
counts.: 1
cases: 1
rules.: 1
in: 1
face: 1
refuse: 1
one--: 1
only: 1
--obvious: 1
it.: 1
obvious: 1
first: 1
often: 1
*right*: 1
it's: 1
it: 1
idea: 1
--: 1
let's: 1
python,: 1
peters: 1
simple: 1
sparse: 1
dense.: 1
aren't: 1
practicality: 1
purity.: 1
pass: 1
silently.: 1
silenced.: 1
ambiguity,: 1
guess.: 1
and: 1
preferably: 1
at: 1
you're: 1
dutch.: 1
good: 1
are: 1
great: 1
more: 1
zen: 1
by: 1
tim: 1
beautiful: 1
explicit: 1
nested.: 1
enough: 1
break: 1
beats: 1
errors: 1
explicitly: 1
temptation: 1
there: 1
that: 1
not: 1
now: 1
never.: 1
now.: 1
hard: 1
bad: 1
easy: 1
namespaces: 1
honking: 1
those!: 1


To gain a thorough understanding of how to scale MapReduce tasks across multiple nodes using Ray,
including memory management, we suggest reading this [insightful blog post on the topic](https://medium.com/distributed-computing-with-ray/executing-adistributed-shuffle-without-a-mapreduce-system-d5856379426c).



## Wrapping up

The important part about this MapReduce example is to realize how flexible Ray’s programming model really is.
Of course, a production-grade MapReduce implementation takes a bit more effort.
But being able to reproduce common algorithms like this one _quickly_ goes a long way.
Keep in mind that in the earlier phases of MapReduce, say around 2010, this paradigm was often the only thing
you had to express your workloads.
With Ray, a whole range of interesting distributed computing patterns
become accessible to any intermediate Python programmer.

If you want to learn more about Ray, and Ray Core and particular, check out the [Ray Core Examples Gallery](./overview.rst),
or some of the ML workloads in our [Use Case Gallery](../../ray-overview/use-cases.rst).
This simple MapReduce example can also be found in ["Learning Ray"](https://maxpumperla.com/learning_ray/),
which contains more examples like this one.